importations

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

data preparation

In [25]:
df = pd.read_csv('./data/urban_development_dataset.csv')
df.head()
df.shape
# print(df.isnull().sum())
# df['development_trend_score'].value_counts()
df.urban_area_type.value_counts()
df.shape
df.head()
le = LabelEncoder()
df['urban_area_type'] = le.fit_transform(df['urban_area_type'])
df.head()
df.describe()

,population_density,median_age,average_household_income,number_of_schools,number_of_hospitals,number_of_parks,public_transport_availability,unemployment_rate,retail_density,office_space_availability,...,air_quality_index,crime_rate_per_1000,fire_station_proximity,proximity_to_highways,internet_speed,housing_price_index,year,road_density,urban_area_type,development_trend_score
count,9500.000000,9500.000000,9.500000e+03,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,...,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,9500.000000,10000.000000,10000.000000
mean,4991.625970,35.279457,6.010906e+04,4.991474,1.982632,3.040842,0.496868,5.018623,50.073169,50109.607293,...,51.040598,46.791214,5.036543,10.062984,50.134666,99.959800,2019.996421,49.871433,1.624300,2.988700
std,1994.854691,9.568304,5.955652e+04,2.227845,1.394126,1.740090,0.287572,1.989349,19.264134,28681.503160,...,20.016145,20.024942,2.883472,5.751247,19.796671,19.825338,1.416440,20.542146,0.707531,1.096491
min,100.000000,18.000000,1.000000e+04,0.000000,0.000000,0.000000,0.000005,0.000000,5.000000,1004.818332,...,0.059818,-7.033524,0.000569,0.003901,1.000000,50.000000,2018.000000,-14.435687,0.000000,1.000000
25%,3640.128794,28.368903,4.129131e+04,3.000000,1.000000,2.000000,0.249202,3.661788,36.888167,25167.483746,...,37.379888,32.924224,2.585283,5.094561,36.550765,86.492808,2019.000000,36.077935,1.000000,2.000000
50%,4984.830208,35.116940,5.511348e+04,5.000000,2.000000,3.000000,0.498003,5.030404,49.714772,50153.479286,...,51.174239,46.518978,5.030731,10.156343,49.912599,99.688797,2020.000000,49.749942,1.000000,3.000000
75%,6330.058011,41.842575,6.874788e+04,6.000000,3.000000,4.000000,0.739598,6.365608,62.785223,75063.433245,...,64.521308,60.544726,7.546573,15.118761,63.506120,113.329764,2021.000000,63.572478,2.000000,4.000000
max,12852.475413,79.790843,1.035413e+06,17.000000,12.000000,11.000000,0.999983,13.596557,129.610619,99997.527825,...,136.233626,127.843821,9.998760,19.998216,122.007397,165.906697,2022.000000,132.515683,3.000000,5.000000


Preprocessing data using median for numerical features | KNN


In [ ]:
# imputer = SimpleImputer(strategy='median') 
# numerical_features = ["population_density",'median_age', 'average_household_income', 'number_of_schools', 'number_of_hospitals','number_of_parks','public_transport_availability' ,'unemployment_rate', 'retail_density','office_space_availability','green_space_percentage','average_temperature','air_quality_index','crime_rate_per_1000','fire_station_proximity','proximity_to_highways','internet_speed','road_density','housing_price_index','year',  ]
# df[numerical_features] = imputer.fit_transform(df[numerical_features])
# df.isnull().sum()

from sklearn.impute import KNNImputer
numerical_features = [
    "population_density", 'median_age', 'average_household_income', 
    'number_of_schools', 'number_of_hospitals', 'number_of_parks', 
    'public_transport_availability', 'unemployment_rate', 'retail_density', 
    'office_space_availability', 'green_space_percentage', 'average_temperature', 
    'air_quality_index', 'crime_rate_per_1000', 'fire_station_proximity', 
    'proximity_to_highways', 'internet_speed', 'road_density', 
    'housing_price_index', 'year'
]

knn_imputer = KNNImputer(n_neighbors=5)  
df[numerical_features] = knn_imputer.fit_transform(df[numerical_features])
print(df.isnull().sum())


Normalisation

In [36]:
correlation_matrix = df.corr()  
threshold = 0.9
correlation_matrix
high_corr_features = set()
for i in range(len(correlation_matrix.columns)):
    for j in range(i):
        if abs(correlation_matrix.iloc[i, j]) > threshold:
            colname = correlation_matrix.columns[i]
            high_corr_features.add(colname)

print(f"Features to drop due to high correlation: {high_corr_features}")
df = df.drop(high_corr_features, axis=1)

Features to drop due to high correlation: {'road_density'}


In [37]:
X = df.drop('development_trend_score', axis=1)
Y = df['development_trend_score']   
scaler = StandardScaler()
X = scaler.fit_transform(X)

check outliers

In [38]:
from scipy.stats import zscore
z_scores = np.abs(zscore(df)) 
outlier_mask = (z_scores > 3).any(axis=1)
plt.figure(figsize=(10, 6))
print(z_scores)

      population_density  median_age  average_household_income  \
0               0.512888    0.756167                  0.029903   
1               0.137358    0.528189                  0.323029   
2               0.667493    0.669652                  0.409018   
3               1.563880    0.085279                  0.108920   
4               0.235552    1.244407                  0.598128   
...                  ...         ...                       ...   
9995            1.336617    0.381161                  0.286295   
9996            2.042161    1.436593                  0.404432   
9997            0.718044    0.371057                  0.260359   
9998            0.511917    1.795142                  0.735555   
9999            0.664113    1.760282                  0.167272   

      number_of_schools  number_of_hospitals  number_of_parks  \
0              0.006087             0.008146         0.031991   
1              0.452271             1.451696         0.031991   
2           

<Figure size 1000x600 with 0 Axes>

In [40]:
# import numpy as np
# import matplotlib.pyplot as plt
# from sklearn.decomposition import PCA

# pca = PCA(n_components=2)
# X_pca = pca.fit_transform(X)

# loadings = pca.components_.T * np.sqrt(pca.explained_variance_)
# plt.figure(figsize=(12, 8))

# plt.scatter(X_pca[:, 0], X_pca[:, 1], c=Y, cmap='viridis', edgecolor='k', s=50)
# plt.colorbar(label='Development Trend Score')
# plt.title('PCA Biplot')
# plt.xlabel('PC1')
# plt.ylabel('PC2')

# for i, var in enumerate(X.T):
#     plt.arrow(0, 0, loadings[i, 0], loadings[i, 1], color='r', alpha=0.7, head_width=0.05, linewidth=1.5)
#     plt.text(loadings[i, 0] * 1.1, loadings[i, 1] * 1.1, var, color='r', ha='center', va='center', fontsize=10)

# plt.grid()
# plt.show()


split data

In [29]:
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.2, random_state=42)
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)
# train_x, train_y = X, Y


(8000, 21) (2000, 21) (8000,) (2000,)


In [39]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

model = svm.SVC(C=1, kernel='linear', degree=3, gamma='auto')
model.fit(train_x, train_y)
y_pred = model.predict(test_x)

# Evaluate the model
print(f"Test Accuracy: {accuracy_score(test_y, y_pred)}")
print(f"Classification Report for SVM:\n{classification_report(test_y, y_pred)}")

Test Accuracy: 0.404
Classification Report for SVM:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       184
           2       0.00      0.00      0.00       387
           3       0.40      1.00      0.58       808
           4       0.00      0.00      0.00       410
           5       0.00      0.00      0.00       211

    accuracy                           0.40      2000
   macro avg       0.08      0.20      0.12      2000
weighted avg       0.16      0.40      0.23      2000



/home/self-ouss/notebooks/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/self-ouss/notebooks/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/self-ouss/notebooks/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is",

grid search

In [45]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
print(f"Performing Cross-Validation for SVM")

param_grid = {
'C': [0.1, 1, 10],  
'kernel': ['linear', 'rbf'],  
'gamma': ['scale', 'auto']  
}
model = svm.SVC()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(train_x, train_y)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best Cross-Validation Score: {grid_search.best_score_}")

best_model = grid_search.best_estimator_
y_pred = best_model.predict(test_x)

# Evaluate the model
print(f"Test Accuracy: {accuracy_score(test_y, y_pred)}")
print(f"Classification Report for SVM:\n{classification_report(test_y, y_pred)}")


Performing Cross-Validation for SVM
Best Parameters: {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'}
Best Cross-Validation Score: 0.401375
Test Accuracy: 0.404
Classification Report for SVM:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00       184
           2       0.00      0.00      0.00       387
           3       0.40      1.00      0.58       808
           4       0.00      0.00      0.00       410
           5       0.00      0.00      0.00       211

    accuracy                           0.40      2000
   macro avg       0.08      0.20      0.12      2000
weighted avg       0.16      0.40      0.23      2000



/home/self-ouss/2cs/ML/tps/ESI_2CS_ML_Labs/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/self-ouss/2cs/ML/tps/ESI_2CS_ML_Labs/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/self-ouss/2cs/ML/tps/ESI_2CS_ML_Labs/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

test data

In [45]:
from sklearn.impute import KNNImputer
df = pd.read_csv('./data/urban_development_test_data.csv')
df.head()
numerical_features = [
    "population_density", 'median_age', 'average_household_income', 
    'number_of_schools', 'number_of_hospitals', 'number_of_parks', 
    'public_transport_availability', 'unemployment_rate', 'retail_density', 
    'office_space_availability', 'green_space_percentage', 'average_temperature', 
    'air_quality_index', 'crime_rate_per_1000', 'fire_station_proximity', 
    'proximity_to_highways', 'internet_speed', 'road_density', 
    'housing_price_index', 'year'
]

knn_imputer = KNNImputer(n_neighbors=5) 
df[numerical_features] = knn_imputer.fit_transform(df[numerical_features])
print(df.isnull().sum())

population_density               0
median_age                       0
average_household_income         0
number_of_schools                0
number_of_hospitals              0
number_of_parks                  0
public_transport_availability    0
unemployment_rate                0
retail_density                   0
office_space_availability        0
green_space_percentage           0
average_temperature              0
air_quality_index                0
crime_rate_per_1000              0
fire_station_proximity           0
proximity_to_highways            0
internet_speed                   0
housing_price_index              0
year                             0
road_density                     0
dtype: int64
